# Reading the Model and adding reactions

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from cobra.io import read_sbml_model
model = read_sbml_model('iML1515.xml.gz')
model

Name,iML1515
Memory address,0x01b06ab2ea30
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


## SucD Model

SucD model is a E.coli model in which 1,4 - butanediol biosynthesis pathway starts from succinate. Activation of succinate to succinyl-CoA is a naturaly occuring reaction, already present in the iML1515 model, so the added reactions start from the reduction step catalyzed by CoA dependent succinate semialdehyde dehydrogenase. Alcohol dehydrogense (last enzyme of the pathway) is present in iML1515 model, but this particular reaction it can catalyze is not, so it was added as last step in biosynthesis pathway. 

In [15]:
model_sucd= model.copy()
model_sucd

Name,iML1515
Memory address,0x01b0757dc6d0
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [16]:
from cobra import Reaction, Metabolite
sucd = Reaction('SUCD') #CoA-dependent succinate semialdehyde dehydrogenase
succinyl_semialdehyde = Metabolite(id='succinyl_semialdehyde_c', compartment='c')

sucd.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              model_sucd.metabolites.succoa_c: -1,
                              succinyl_semialdehyde: 1, #new one
                              model_sucd.metabolites.coa_c: 1,
                              model_sucd.metabolites.nad_c: 1
                             })
print(sucd.build_reaction_string())
model_sucd.add_reactions([sucd])
model_sucd.reactions.SUCD

nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c


Reaction identifier,SUCD
Name,
Memory address,0x01b075efdcd0
Stoichiometry,nadh_c + succoa_c --> coa_c + h_c + nad_c + succinyl_semialdehyde_c Nicotinamide adenine dinucleotide - reduced + Succinyl-CoA --> Coenzyme A + H+ + Nicotinamide adenine dinucleotide +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [17]:
from cobra import Reaction, Metabolite
fhbd = Reaction('FHBD') #4-hydroxybutyrate dehydrogenase
four_hydroxybutyrate = Metabolite(id='four_hydroxybutyrate_c', compartment='c')

fhbd.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              succinyl_semialdehyde: -1, 
                              four_hydroxybutyrate: 1, #new one
                              model_sucd.metabolites.nad_c: 1
                             })
print(fhbd.build_reaction_string())
model_sucd.add_reactions([fhbd])
model_sucd.reactions.FHBD

nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c


Reaction identifier,FHBD
Name,
Memory address,0x01b075efdd90
Stoichiometry,nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c Nicotinamide adenine dinucleotide - reduced + --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [18]:
from cobra import Reaction, Metabolite
hbd_coa_t = Reaction('HBCT') #4-hydroxybutyryl-CoA transferase
four_hbcoa = Metabolite(id='four_hbcoa_c', compartment='c')
hbd_coa_t.add_metabolites({model_sucd.metabolites.accoa_c: -1,
                              model_sucd.metabolites.ac_c: 1,
                              four_hbcoa: 1, #new one
                              four_hydroxybutyrate: -1
                             })
print(hbd_coa_t.build_reaction_string())
model_sucd.add_reactions([hbd_coa_t])
model_sucd.reactions.HBCT

accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c


Reaction identifier,HBCT
Name,
Memory address,0x01b075effd00
Stoichiometry,accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c Acetyl-CoA + --> Acetate +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [19]:
from cobra import Reaction, Metabolite
hbd_coa_r = Reaction('HBCR') #4-hydroxybutyryl-CoA reductase
four_hbaldehyde = Metabolite(id='four_hbaldehyde_c', compartment='c')
hbd_coa_r.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              four_hbaldehyde: 1, #new one
                              four_hbcoa: -1,
                              model_sucd.metabolites.nad_c: 1,
                              model_sucd.metabolites.coa_c: 1
                             })
print(hbd_coa_r.build_reaction_string())
model_sucd.add_reactions([hbd_coa_r])
model_sucd.reactions.HBCR

four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c


Reaction identifier,HBCR
Name,
Memory address,0x01b075eff400
Stoichiometry,four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> Coenzyme A + + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [20]:
from cobra import Reaction, Metabolite
adh= Reaction('ADH') #alcohol dehydrogenase
bdo = Metabolite(id='bdo_c', compartment='c')
adh.add_metabolites({model_sucd.metabolites.h_c: 1,
                              model_sucd.metabolites.nadh_c: -1,
                              four_hbaldehyde: -1,
                              bdo: 1, #new one
                              model_sucd.metabolites.nad_c: 1,
                             })
print(adh.build_reaction_string())
model_sucd.add_reactions([adh])
model_sucd.reactions.ADH

four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c


Reaction identifier,ADH
Name,
Memory address,0x01b075effe20
Stoichiometry,four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [21]:
bdo_exchange = Reaction('EX_bdo')
bdo_exchange.add_metabolites({model_sucd.metabolites.bdo_c: -1})
model_sucd.add_reaction(bdo_exchange)
model_sucd.add_boundary(model_sucd.metabolites.bdo_c, type='demand')

Reaction identifier,DM_bdo_c
Name,demand
Memory address,0x01b075ef0f40
Stoichiometry,bdo_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


## Akg Model

Akg model SucD is a E.coli model in which 1,4 - butanediol biosynthesis pathway starts from alpha - ketoglutarate. All of the reactions in this pathway are not native to E.coli, with the exception of naturally occuring last enzyme (alcohol dehydrogenase). While the enzyme is present in the cells, for the model to work 4-hydroxybutyraldehyde (substrate) and 1,4 - butanediol (product) needed to be connected to this enzyme. 

In [9]:
model_akg=model.copy()
model_akg

Name,iML1515
Memory address,0x01a5b12fadc0
Number of metabolites,1877
Number of reactions,2712
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [10]:
from cobra import Reaction, Metabolite
oxodc = Reaction('OXODC') #2-oxoglutarate decarboxylase
succinyl_semialdehyde = Metabolite(id='succinyl_semialdehyde_c', compartment='c')

oxodc.add_metabolites({model_akg.metabolites.akg_c: -1,
                              succinyl_semialdehyde: 1, #new one
                              model_akg.metabolites.co2_c: 1
                             })
print(oxodc.build_reaction_string())
model_akg.add_reactions([oxodc])
model_akg.reactions.OXODC

akg_c --> co2_c + succinyl_semialdehyde_c


Reaction identifier,OXODC
Name,
Memory address,0x01a5b1dd9d90
Stoichiometry,akg_c --> co2_c + succinyl_semialdehyde_c 2-Oxoglutarate --> CO2 CO2 +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [11]:
from cobra import Reaction, Metabolite
fhbd = Reaction('FHBD') #4-hydroxybutyrate dehydrogenase
four_hydroxybutyrate = Metabolite(id='four_hydroxybutyrate_c', compartment='c')

fhbd.add_metabolites({model_akg.metabolites.h_c: 1,
                              model_akg.metabolites.nadh_c: -1,
                              succinyl_semialdehyde: -1, 
                              four_hydroxybutyrate: 1, #new one
                              model_akg.metabolites.nad_c: 1
                             })
print(fhbd.build_reaction_string())
model_akg.add_reactions([fhbd])
model_akg.reactions.FHBD

nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c


Reaction identifier,FHBD
Name,
Memory address,0x01a5b1dd9a90
Stoichiometry,nadh_c + succinyl_semialdehyde_c --> four_hydroxybutyrate_c + h_c + nad_c Nicotinamide adenine dinucleotide - reduced + --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [12]:
from cobra import Reaction, Metabolite
hbd_coa_t = Reaction('HBCT') #4-hydroxybutyryl-CoA transferase
four_hbcoa = Metabolite(id='four_hbcoa_c', compartment='c')
hbd_coa_t.add_metabolites({model_akg.metabolites.accoa_c: -1,
                              model_akg.metabolites.ac_c: 1,
                              four_hbcoa: 1, #new one
                              four_hydroxybutyrate: -1
                             })
print(hbd_coa_t.build_reaction_string())
model_akg.add_reactions([hbd_coa_t])
model_akg.reactions.HBCT

accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c


Reaction identifier,HBCT
Name,
Memory address,0x01a5b1ddde80
Stoichiometry,accoa_c + four_hydroxybutyrate_c --> ac_c + four_hbcoa_c Acetyl-CoA + --> Acetate +
GPR,
Lower bound,0.0
Upper bound,1000.0


In [13]:
from cobra import Reaction, Metabolite
hbd_coa_r = Reaction('HBCR') #4-hydroxybutyryl-CoA reductase
four_hbaldehyde = Metabolite(id='four_hbaldehyde_c', compartment='c')
hbd_coa_r.add_metabolites({model_akg.metabolites.h_c: 1,
                              model_akg.metabolites.nadh_c: -1,
                              four_hbaldehyde: 1, #new one
                              four_hbcoa: -1,
                              model_akg.metabolites.nad_c: 1,
                              model_akg.metabolites.coa_c: 1
                             })
print(hbd_coa_r.build_reaction_string())
model_akg.add_reactions([hbd_coa_r])
model_akg.reactions.HBCR

four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c


Reaction identifier,HBCR
Name,
Memory address,0x01a5b1dd9b50
Stoichiometry,four_hbcoa_c + nadh_c --> coa_c + four_hbaldehyde_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> Coenzyme A + + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [14]:
from cobra import Reaction, Metabolite
adh= Reaction('ADH') #alcohol dehydrogenase
bdo = Metabolite(id='bdo_c', compartment='c')
adh.add_metabolites({model_akg.metabolites.h_c: 1,
                              model_akg.metabolites.nadh_c: -1,
                              four_hbaldehyde: -1,
                              bdo: 1, #new one
                              model_akg.metabolites.nad_c: 1,
                             })
print(adh.build_reaction_string())
model_akg.add_reactions([adh])
model_akg.reactions.ADH

four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c


Reaction identifier,ADH
Name,
Memory address,0x01a5b1dd9c40
Stoichiometry,four_hbaldehyde_c + nadh_c --> bdo_c + h_c + nad_c + Nicotinamide adenine dinucleotide - reduced --> + H+ + Nicotinamide adenine dinucleotide
GPR,
Lower bound,0.0
Upper bound,1000.0


In [15]:
bdo_exchange = Reaction('EX_bdo')
bdo_exchange.add_metabolites({model_akg.metabolites.bdo_c: -1})
model_akg.add_reaction(bdo_exchange)
model_akg.add_boundary(model_akg.metabolites.bdo_c, type='demand')

Reaction identifier,DM_bdo_c
Name,demand
Memory address,0x01a5b1dddb50
Stoichiometry,bdo_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0
